VERSI 2
PER BATCH

In [20]:
import pandas as pd
import json
import math
import re
import random
import os
from openai import OpenAI

In [ ]:
# ============================================
# KONFIGURASI
# ============================================

client = OpenAI(api_key="API-KEY")
# Cache sederhana untuk menghindari panggilan API berulang
_gen_cache = {}

DRIVE_DATASET_DIR = "PATH_TO_DIRECTORY/dtp-data-pipeline/Pipeline Flagging/Data Diploy Flagged/Flagged_1000_Per_Class"  # sesuaikan dengan struktur folder penyimpanan data

INPUT_PATH = f"{DRIVE_DATASET_DIR}/Layanan_Teknologi_Informasi_7_0.xlsx" # SESUAIKAN DENGAN FILE EXCEL YANG MAU DIPROSES
OUTPUT_PREFIX = f"{DRIVE_DATASET_DIR}/dataset_multiturn/Layanan_Teknologi_Informasi_7_0"  # nanti jadi ..._batchXXX.jsonl

BATCH_SIZE = 1000  # jumlah row per batch file
PROCESS_LIMIT = None  # None = semua row; atau angka misal 1000

COLUMN_MAP = {
    "jenjang": "jenjang",
    "jurusan": "program_studi",
    "deskripsi_studi": "deskripsi_studi",

    "nama_pelatihan": "Nama_Pelatihan",
    "tema_pelatihan": "Pelatihan_Tema",

    "judul_sertifikasi": "Sertifikasi",
    "bidang_sertifikasi": "Bidang_Sertifikasi",

    "bidang_pekerjaan": "bidang_pekerjaan",
    "jabatan_terakhir": "Pekerjaan_Posisi",

    "lama_bekerja": "Lama Bekerja",
    "keterampilan": "keterampilan"
}

In [31]:
def normalize_text(text: str) -> str:
    """
    Normalisasi jawaban agar:
    - Tidak mengandung heading Markdown (###, ####, dst.)
    - Tidak mengandung bullet Markdown (- , * )
    - Newline berlebihan dirapikan jadi paragraf natural.
    """
    if text is None:
        return ""
    t = str(text).strip()

    # Pecah per baris dan bersihkan heading / bullet di awal baris
    lines = []
    for line in t.splitlines():
        line = line.lstrip()  # hapus spasi awal
        # Hapus heading Markdown (#, ##, ###, dst.)
        if line.startswith("#"):
            line = line.lstrip("#").strip()
        # Hapus bullet Markdown (- , * ) di awal baris
        if line.startswith("- ") or line.startswith("* "):
            line = line[2:].strip()
        if line:
            lines.append(line)

    # Gabungkan lagi jadi satu paragraf panjang
    t = " ".join(lines)
    # Rapatkan spasi berlebih
    t = re.sub(r"\s+", " ", t).strip()
    return t

In [32]:
def strip_redundant_prefix(text: str, phrases) -> str:
    """
    Menghapus frasa pembuka yang berulang / tidak diinginkan di awal jawaban.
    """
    if text is None:
        return ""
    t = text.strip()
    if isinstance(phrases, str):
        phrases = [phrases]

    for p in phrases:
        pattern = r"^" + re.escape(p) + r"[\s,:-]*"
        t = re.sub(pattern, "", t, flags=re.I)

    return t.strip()

In [33]:
def gen(prompt, max_tokens=150):
    """
    Wrapper pemanggilan API:
    - Menambahkan aturan anti-Markdown
    - Mengembalikan teks yang sudah dinormalisasi.
    - Menggunakan cache untuk prompt yang sama.
    """
    key = (prompt, max_tokens)
    if key in _gen_cache:
        return _gen_cache[key]

    full_prompt = (
        prompt
        + "\n\n"
        + "Batasan format jawaban:\n"
          "- Jawab dalam bentuk paragraf natural.\n"
          "- DILARANG menggunakan Markdown (tidak boleh ###, ####, bullet list, atau heading).\n"
          "- Jangan gunakan tanda '-' atau '*' di awal baris.\n"
          "- Jangan buat list bernomor.\n"
          "- Jawaban maksimal 3 kalimat."
    )

    resp = client.chat.completions.create(
        model="gpt-4o-mini", #BISA DIGANTI MODEL OPEN AI LAINNYA
        messages=[{"role": "user", "content": full_prompt}],
        max_tokens=max_tokens,
        temperature=0.7
    )
    raw = resp.choices[0].message.content
    out = normalize_text(raw)
    _gen_cache[key] = out
    return out

In [34]:
def get_val(row, key):
    col = COLUMN_MAP.get(key)
    if col not in row.index:
        return ""
    val = row[col]
    if isinstance(val, float) and math.isnan(val):
        return ""
    if val is None:
        return ""
    return str(val).strip()

In [35]:
def is_empty(x):
    if x is None:
        return True
    x = str(x).strip()
    return x == "" or x.lower() in ["nan", "none", "null", "-"]

In [36]:
def build_intro_template(
    jenjang, jurusan, jabatan, bidang_pekerjaan,
    pelatihan, sertifikasi, lama_bekerja, keterampilan
):
    """
    Intro versi template: 'Berikut data awal profil saya: ...'
    Dipakai untuk N data awal (misal INTRO_TEMPLATE_LIMIT).
    """
    summary = []

    if not is_empty(jenjang) or not is_empty(jurusan):
        summary.append(f"Pendidikan: {jenjang} {jurusan}".strip())

    if not is_empty(pelatihan):
        summary.append(f"Pelatihan: {pelatihan}")

    if not is_empty(sertifikasi):
        summary.append(f"Sertifikasi: {sertifikasi}")

    if not is_empty(jabatan) or not is_empty(bidang_pekerjaan):
        if is_empty(bidang_pekerjaan):
            summary.append(f"Pekerjaan: {jabatan}")
        else:
            summary.append(f"Pekerjaan: {jabatan} di bidang {bidang_pekerjaan}")

    if not is_empty(lama_bekerja):
        summary.append(f"Lama bekerja: {lama_bekerja}")

    if not is_empty(keterampilan):
        summary.append(f"Keterampilan: {keterampilan}")

    if summary:
        return "Berikut data awal profil saya: " + "; ".join(summary)
    else:
        return "Saya ingin melengkapi data awal profil saya di platform Diploy."

In [37]:
def build_intro(
    jenjang, jurusan, jabatan, bidang_pekerjaan,
    pelatihan, sertifikasi, lama_bekerja, keterampilan
):
    """
    Intro dinamis via API, natural, seperti ngobrol ke HR.
    Dipakai untuk data setelah INTRO_TEMPLATE_LIMIT.
    """
    info_list = []

    if not is_empty(jenjang) or not is_empty(jurusan):
        info_list.append(f"Pendidikan: {jenjang} {jurusan}".strip())

    if not is_empty(jabatan) or not is_empty(bidang_pekerjaan):
        if is_empty(bidang_pekerjaan):
            info_list.append(f"Pekerjaan: {jabatan}")
        else:
            info_list.append(f"Pekerjaan: {jabatan} di bidang {bidang_pekerjaan}")

    if not is_empty(pelatihan):
        info_list.append(f"Pelatihan: {pelatihan}")

    if not is_empty(sertifikasi):
        info_list.append(f"Sertifikasi: {sertifikasi}")

    if not is_empty(lama_bekerja):
        info_list.append(f"Lama bekerja: {lama_bekerja}")

    if not is_empty(keterampilan):
        info_list.append(f"Keterampilan: {keterampilan}")

    merged_info = "; ".join(info_list)

    if merged_info.strip() == "":
        prompt = """
        Buatkan kalimat pembuka percakapan seperti seorang kandidat yang ingin mengisi profilnya
        di platform diploy pencarian kerja, tanpa menyebut detail spesifik karena datanya belum lengkap.

        Gunakan Bahasa Indonesia natural dan profesional, bukan kaku.
        Maksimal 2 kalimat, tanpa Markdown.
        """
        return gen(prompt, max_tokens=80)

    prompt = f"""
    Buatkan kalimat pembuka percakapan seperti memberikan informasi dasar terkait latar belakang pendidikan, pelatihan yang diikuti, sertifikasi dan pekerjaan.

    Gunakan Bahasa Indonesia natural dan profesional, bukan formal kaku.

    Informasi yang tersedia:
    {merged_info}

    Ketentuan:
    - Maksimal 2–3 kalimat.
    - Hindari pola tetap seperti “Berikut data awal profil saya”.
    - Variasikan gaya penulisan seperti manusia yang sedang bertutur.
    - DILARANG menggunakan Markdown, bullet list, atau heading.
    - Jawaban harus berupa paragraf natural.
    """

    intro = gen(prompt, max_tokens=120)
    return intro

In [38]:
df = pd.read_excel(INPUT_PATH)

total_rows = len(df)
if PROCESS_LIMIT is not None:
    effective_rows = min(total_rows, PROCESS_LIMIT)
else:
    effective_rows = total_rows

INTRO_TEMPLATE_LIMIT = 4000  # berapa data awal pakai "Berikut data awal profil saya"

print(f"Total baris di Excel      : {total_rows}")
print(f"Effective rows (diproses) : {effective_rows}")
print(f"BATCH_SIZE                : {BATCH_SIZE}")

# Hitung jumlah batch
num_batches = (effective_rows + BATCH_SIZE - 1) // BATCH_SIZE
print(f"Total batch               : {num_batches}")

for batch_idx in range(num_batches):
    start = batch_idx * BATCH_SIZE
    end = min(start + BATCH_SIZE, effective_rows)

    batch_file = f"{OUTPUT_PREFIX}_batch{batch_idx+1:03d}.jsonl"

    # Kalau file batch sudah ada, skip (anggap sudah selesai)
    if os.path.exists(batch_file):
        print(f"[SKIP] {batch_file} sudah ada. Lewati batch {batch_idx+1} (rows {start}-{end-1}).")
        continue

    print(f"[RUN ] Batch {batch_idx+1}/{num_batches} -> rows {start} s.d. {end-1}")
    print(f"       Output: {batch_file}")

    with open(batch_file, "w", encoding="utf-8") as f_out:
        for idx in range(start, end):
            row = df.iloc[idx]

            if idx % 100 == 0:
                print(f"  - Processed row {idx}/{effective_rows}")

            jenjang            = get_val(row, "jenjang")
            jurusan            = get_val(row, "jurusan")
            deskripsi_studi    = get_val(row, "deskripsi_studi")

            nama_pelatihan     = get_val(row, "nama_pelatihan")
            tema_pelatihan     = get_val(row, "tema_pelatihan")

            judul_sertifikasi  = get_val(row, "judul_sertifikasi")
            bidang_sertifikasi = get_val(row, "bidang_sertifikasi")

            bidang_pekerjaan   = get_val(row, "bidang_pekerjaan")
            jabatan_terakhir   = get_val(row, "jabatan_terakhir")

            lama_bekerja       = get_val(row, "lama_bekerja")
            keterampilan       = get_val(row, "keterampilan")

            ada_pendidikan = not (is_empty(jenjang) and is_empty(jurusan))
            ada_pelatihan  = not (is_empty(nama_pelatihan) and is_empty(tema_pelatihan))
            ada_sertif     = not (is_empty(judul_sertifikasi) and is_empty(bidang_sertifikasi))
            ada_pekerjaan  = not (is_empty(bidang_pekerjaan) and is_empty(jabatan_terakhir) and is_empty(lama_bekerja))

            messages = []

            # ---------- SYSTEM ----------
            messages.append({
                "role": "system",
                "content": "Kamu asisten asesmen Diploy. Tugasmu memetakan profil user ke area fungsi dan level okupasi PON-TIK. Jawab dengan Bahasa Indonesia formal, ringkas, dan terstruktur."
            })

            # ---------- USER INTRO ----------
            if idx < INTRO_TEMPLATE_LIMIT:
                intro_text = build_intro_template(
                    jenjang, jurusan, jabatan_terakhir, bidang_pekerjaan,
                    nama_pelatihan, judul_sertifikasi, lama_bekerja, keterampilan
                )
            else:
                intro_text = build_intro(
                    jenjang, jurusan, jabatan_terakhir, bidang_pekerjaan,
                    nama_pelatihan, judul_sertifikasi, lama_bekerja, keterampilan
                )

            messages.append({"role": "user", "content": intro_text})

            if not ada_pendidikan:
                messages.append({
                    "role": "assistant",
                    "content": "Apa jenjang pendidikan dan program studi terakhir yang Anda tempuh?"
                })

                ans = gen(
                    "Buatkan jawaban manusia untuk pertanyaan tentang pendidikan terakhir, "
                    "tanpa data di spreadsheet. Jawaban profesional namun tetap realistis, "
                    "Bahasa Indonesia formal, maksimal 2 kalimat. "
                    "Jawaban gunakan sudut pandang orang pertama (saya)."
                )
                messages.append({"role": "user", "content": ans})

                messages.append({
                    "role": "assistant",
                    "content": "Bisakah dijelaskan apa yang Anda pelajari selama masa studi Anda?"
                })

                body = gen(
                    "Buat penjelasan singkat tentang apa yang umumnya dipelajari selama pendidikan formal sesuai data pendidikan yang dimiliki. "
                    "Jawab dari sudut pandang orang pertama, tetapi JANGAN mulai dengan frasa "
                    "'Yang saya pelajari selama menempuh pendidikan adalah'."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Yang saya pelajari selama menempuh pendidikan adalah",
                        "yang saya pelajari selama menempuh pendidikan adalah",
                        "dalam perjalanan studi saya, "
                    ]
                )
                ans = "Yang saya pelajari selama menempuh pendidikan adalah " + body
                messages.append({"role": "user", "content": ans})

                messages.append({
                    "role": "assistant",
                    "content": "Keterampilan apa saja yang didapat selama menjalani pendidikan?"
                })

                body = gen(
                    "LENGKAPI bagian setelah frasa 'Saya memiliki keterampilan ' dengan beberapa keterampilan umum "
                    "yang biasanya diperoleh dari pendidikan formal, misalnya 'kemampuan berpikir kritis, pemecahan masalah, ...'. "
                    "Jawaban harus LANGSUNG dimulai dengan menyebut keterampilan, bukan dengan frasa pembuka seperti "
                    "'dalam perjalanan saya', 'saya juga', atau 'selain itu'. "
                    "JANGAN mengulang frasa 'Saya memiliki keterampilan'."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Saya memiliki keterampilan",
                        "Dalam perjalanan studi",
                        "Dalam perjalanan studi saya",
                        "Selain itu,",
                        "Selain itu"
                    ]
                )
                ans = "Saya memiliki keterampilan " + body
                messages.append({"role": "user", "content": ans})

            else:
                messages.append({
                    "role": "assistant",
                    "content": "Bisakah dijelaskan apa yang Anda pelajari selama masa studi Anda?"
                })

                if is_empty(deskripsi_studi):
                    if not is_empty(jurusan):
                        prompt = (
                            f"LENGKAPI isi setelah kata 'adalah' dari kalimat berikut:\n"
                            f"'Yang saya pelajari selama menempuh pendidikan di jenjang {jenjang} "
                            f"program studi {jurusan} adalah ...'\n\n"
                            f"Jawab dari sudut pandang orang pertama. "
                            f"JANGAN mengulang frasa pembuka tersebut dan JANGAN mengulang kata 'di jenjang' atau 'program studi' "
                            f"di awal jawaban Anda. Jika tidak yakin dengan detail spesifik kurikulumnya, "
                            f"berikan jawaban umum yang tetap relevan dengan bidang {jurusan}."
                        )
                        body = gen(prompt)
                        body = strip_redundant_prefix(
                            body,
                            [
                                "yang saya pelajari",
                                "di jenjang",
                                "pada jenjang",
                                "di program studi",
                                "pada program studi"
                            ]
                        )
                        ans = (
                            f"Yang saya pelajari selama menempuh pendidikan di jenjang {jenjang} "
                            f"program studi {jurusan} adalah {body}"
                        )
                    else:
                        prompt = (
                            "LENGKAPI isi setelah kata 'adalah' dari kalimat:\n"
                            "'Yang saya pelajari selama menempuh pendidikan adalah ...'\n\n"
                            "Jawab dari sudut pandang orang pertama, dengan penjelasan umum yang sesuai "
                            "dengan pendidikan formal. JANGAN mengulang frasa pembuka tersebut."
                        )
                        body = gen(prompt)
                        body = strip_redundant_prefix(
                            body,
                            [
                                "yang saya pelajari",
                                "di jenjang",
                                "pada jenjang",
                                "di program studi",
                                "pada program studi"
                            ]
                        )
                        ans = "Yang saya pelajari selama menempuh pendidikan adalah " + body
                else:
                    deskripsi_bersih = normalize_text(deskripsi_studi)
                    if deskripsi_bersih.lower().startswith("yang saya pelajari"):
                        ans = deskripsi_bersih
                    else:
                        if not is_empty(jurusan):
                            ans = (
                                f"Yang saya pelajari selama menempuh pendidikan di jenjang {jenjang} "
                                f"program studi {jurusan} adalah {deskripsi_bersih}"
                            )
                        else:
                            ans = "Yang saya pelajari selama menempuh pendidikan adalah " + deskripsi_bersih

                messages.append({"role": "user", "content": ans})

                messages.append({
                    "role": "assistant",
                    "content": "Keterampilan apa saja yang didapat selama menjalani pendidikan?"
                })

                body = gen(
                    f"LENGKAPI bagian setelah frasa 'Saya memiliki keterampilan ' dengan beberapa keterampilan umum "
                    f"dan keterampilan khusus yang relevan dengan jurusan {jurusan}. "
                    f"Jawaban harus LANGSUNG menyebutkan keterampilan, misalnya "
                    f"'pemrograman dasar, analisis data, dan kemampuan bekerja dalam tim', tanpa frasa pembuka seperti "
                    f"'dalam perjalanan saya', 'saya juga', atau 'selain itu'. "
                    f"JANGAN mengulang frasa 'Saya memiliki keterampilan'. "
                    f"Jika nama jurusan tidak jelas, berikan keterampilan umum yang tetap relevan dan tidak menyimpang."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Saya memiliki keterampilan",
                        "Dalam perjalanan",
                        "Dalam perjalanan saya",
                        "Selain itu,",
                        "Selain itu"
                    ]
                )
                ans = "Saya memiliki keterampilan " + body
                messages.append({"role": "user", "content": ans})

            if not ada_pelatihan:
                tanya_pelatihan_1 = [
                    "Sekarang saya ingin mengetahui pengalaman pelatihan Anda. Pelatihan apa saja yang pernah Anda ikuti?",
                    "Setelah memahami riwayat pendidikan Anda, apakah Anda juga pernah mengikuti pelatihan tertentu? Jika ya, sebutkan pelatihan tersebut.",
                    "Berikutnya, saya ingin menanyakan mengenai pelatihan yang pernah Anda ikuti. Bisa dijelaskan pelatihan apa saja?",
                    "Untuk melengkapi data, apakah ada pelatihan yang pernah Anda ikuti sebelumnya?"
                ]
                messages.append({
                    "role": "assistant",
                    "content": random.choice(tanya_pelatihan_1)
                })

                ans = gen(
                    f"Buat jawaban realistis tentang pelatihan umum yang mungkin pernah diikuti seseorang "
                    f"dengan latar belakang {jurusan}. "
                    f"Jika tidak ada pelatihan formal, jelaskan bahwa belum pernah mengikuti pelatihan terstruktur, "
                    f"namun boleh menyebut belajar mandiri jika relevan."
                )
                messages.append({"role": "user", "content": ans})

                tanya_pelatihan_2 = [
                    "Berdasarkan pelatihan yang Anda sebutkan, keterampilan apa saja yang Anda dapatkan selama mengikuti pelatihan tersebut?",
                    "Dari pelatihan tersebut, kemampuan atau keterampilan apa yang paling Anda rasakan berkembang?",
                    "Menurut Anda, keterampilan apa saja yang berhasil Anda bangun selama mengikuti pelatihan itu?"
                ]

                messages.append({
                    "role": "assistant",
                    "content": random.choice(tanya_pelatihan_2)
                })

                body = gen(
                    "Buat jawaban dalam bentuk paragraf dari sudut pandang orang pertama tentang keterampilan "
                    "yang diperoleh dari pelatihan yang telah disebutkan. "
                    "JANGAN mulai jawaban dengan frasa 'Saya mendapatkan keterampilan'."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Saya mendapatkan keterampilan",
                        "Dalam pelatihan ini",
                        "Dalam pelatihan tersebut"
                    ]
                )

                ans = "Saya mendapatkan keterampilan " + body
                messages.append({"role": "user", "content": ans})
            else:
                tanya_pelatihan_isi = [
                    "Terkait pelatihan yang Anda ikuti, apa saja yang Anda pelajari selama mengikuti pelatihan?",
                    "Dari pelatihan yang Anda ikuti, materi atau topik apa yang paling banyak Anda pelajari?",
                    "Terkait pelatihan yang Anda ikuti, apa saja hal utama yang Anda pelajari dari pelatihan tersebut?"
                ]
                messages.append({
                    "role": "assistant",
                    "content": random.choice(tanya_pelatihan_isi)
                })

                body = gen(
                    f"Jelaskan secara singkat dari sudut pandang orang pertama apa yang dipelajari dari pelatihan {nama_pelatihan} "
                    f"dengan tema {tema_pelatihan}. "
                    f"JANGAN mulai jawaban dengan frasa 'Yang saya pelajari dari pelatihan tersebut adalah'. "
                    f"Jika tema kurang jelas, berikan penjelasan umum yang tetap relevan."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Yang saya pelajari dari pelatihan tersebut adalah",
                        "Yang saya pelajari dari pelatihan ini adalah"
                    ]
                )
                ans = "Yang saya pelajari dari pelatihan tersebut adalah " + body
                messages.append({"role": "user", "content": ans})

                tanya_pelatihan_skill = [
                    "Berdasarkan pelatihan yang Anda sebutkan sebelumnya, keterampilan apa saja yang Anda dapatkan selama mengikuti pelatihan?",
                    "Menurut Anda,  keterampilan apa yang paling berkembang setelah mengikuti pelatihan?",
                    "Menurut Anda, kompetensi atau keterampilan apa saja yang terbentuk setelah Anda mengikuti pelatihan?"
                ]
                messages.append({
                    "role": "assistant",
                    "content": random.choice(tanya_pelatihan_skill)
                })

                body = gen(
                    f"Jelaskan secara naratif dari sudut pandang orang pertama beberapa keterampilan yang logis "
                    f"didapatkan dari pelatihan {nama_pelatihan} yang bertema {tema_pelatihan}. "
                    f"JANGAN mulai jawaban dengan frasa 'Saya mendapatkan keterampilan'."
                )
                body = strip_redundant_prefix(
                    body,
                    [
                        "Saya mendapatkan keterampilan",
                        "Dalam pelatihan ini",
                        "Dalam pelatihan tersebut"
                    ]
                )
                ans = "Saya mendapatkan keterampilan " + body
                messages.append({"role": "user", "content": ans})

            if ada_pelatihan and ada_pekerjaan:
                tanya_relasi = [
                    "Dari pelatihan yang Anda ikuti, mana yang menurut Anda paling relevan dengan pekerjaan Anda saat ini? berikan alasan singkatnya",
                    "Menurut Anda, pelatihan apa yang paling membantu dalam menjalankan pekerjaan Anda sekarang?",
                    "Jika dibandingkan dengan pekerjaan Anda, pelatihan mana yang dirasa paling mendukung tugas sehari-hari?"
                ]
                messages.append({
                    "role": "assistant",
                    "content": random.choice(tanya_relasi)
                })

                ans = gen(
                    f"Buat jawaban yang mengaitkan pelatihan {nama_pelatihan} dengan pekerjaan {jabatan_terakhir} "
                    f"di bidang {bidang_pekerjaan}."
                )
                messages.append({"role": "user", "content": ans})

            if not ada_sertif:
                messages.append({
                    "role": "assistant",
                    "content": "Sebutkan sertifikasi dibidang apa saja yang Anda miliki?"
                })

                ans = gen(
                    f"Buat jawaban bahwa user belum memiliki sertifikasi formal, "
                    f"namun sebutkan 1–2 sertifikasi yang paling ingin diambil berdasarkan jurusan {jurusan} "
                    f"atau bidang {bidang_pekerjaan}."
                )
                messages.append({"role": "user", "content": ans})

            if not ada_pekerjaan:
              # 5.a Tidak ada data pekerjaan di spreadsheet
              # Tanya apakah punya pekerjaan saat ini
                messages.append({
                  "role": "assistant",
                  "content": "Apakah Anda memiliki pekerjaan saat ini?"
                })

                ans = gen(
                  "Buat jawaban seseorang yang belum bekerja full-time namun mungkin pernah magang, freelance, "
                  "atau baru saja menyelesaikan kontrak kerja. Jawaban gunakan sudut pandang orang pertama (saya)."
                )
                messages.append({"role": "user", "content": ans})

              # Lanjut: tanya durasi / lama pengalaman tersebut (jika ada)
                tanya_lama = [
                  "Jika Anda pernah bekerja atau magang, berapa lama pengalaman tersebut?",
                  "Berapa lama Anda menjalani pekerjaan atau pengalaman kerja yang Anda sebutkan tadi?",
                  "Bila Anda memiliki pengalaman kerja, sudah berapa lama Anda menjalaninya?"
                ]
                messages.append({
                  "role": "assistant",
                  "content": random.choice(tanya_lama)
                })

                ans = gen(
                  "Buat jawaban lama bekerja yang realistis untuk seseorang yang belum memiliki pekerjaan tetap, "
                  "namun bisa saja pernah magang atau freelance. Jika tidak ada pengalaman kerja sama sekali, "
                  "jelaskan bahwa belum memiliki pengalaman kerja formal."
                )
                messages.append({"role": "user", "content": ans})

              # Lanjut: tanya aktivitas/tanggung jawab harian terkait pekerjaan/pengalaman
                tanya_jobdesk = [
                  "Dari pengalaman yang Anda miliki, apa saja tugas atau aktivitas utama yang biasanya Anda kerjakan?",
                  "Bisa diceritakan secara singkat tugas atau aktivitas harian yang pernah Anda jalani dalam pekerjaan atau magang tersebut?",
                  "Jika Anda pernah bekerja atau magang, apa saja tanggung jawab utama yang biasa Anda lakukan?"
                ]
                messages.append({
                  "role": "assistant",
                  "content": random.choice(tanya_jobdesk)
                })

                ans = gen(
                  "Buat deskripsi singkat aktivitas harian berdasarkan pengalaman kerja/magang yang umum, "
                  "misalnya membantu administrasi, mengolah data, mendukung tim, atau tugas teknis sederhana. "
                  "Jika sebelumnya digambarkan belum pernah bekerja, jelaskan aktivitas rutin dalam mengembangkan diri "
                  "seperti belajar mandiri, mengikuti pelatihan, atau kegiatan produktif lainnya."
                )
                messages.append({"role": "user", "content": ans})

            else:
              # 5.b Ada data pekerjaan di spreadsheet → langsung tanya jobdesk
                tanya_jobdesk = [
                  "Bisakah dijelaskan secara singkat tanggung jawab harian Anda pada pekerjaan saat ini?",
                  "Apa saja tugas yang biasanya Anda kerjakan dalam pekerjaan Anda?",
                  "Secara umum, apa saja aktivitas dan tugas rutin harian pada pekerjaan Anda?",
                  "Dapatkah Anda menceritakan tugas atau peran harian Anda di pekerjaan yang saat ini Anda lakukan?",
                  "Berdasarkan pekerjaan yang Anda sebutkan sebelumnya, apa saja tanggung jawab utama Anda setiap hari?"
                ]
                messages.append({
                  "role": "assistant",
                  "content": random.choice(tanya_jobdesk)
                })

                ans = gen(
                  f"Buat deskripsi tugas harian peran {jabatan_terakhir} di bidang {bidang_pekerjaan}. "
                  f"Jawaban gunakan sudut pandang orang pertama (saya), dan jelaskan 3–5 aktivitas utama yang relevan."
                )
                messages.append({"role": "user", "content": ans})

            messages.append({
                "role": "assistant",
                "content":
                    "Terima kasih, informasi Anda sudah lengkap. Selanjutnya saya akan memetakan profil Anda ke okupasi dan level yang sesuai." #UPDATE JADI ADA AREA FUNGSI DAN LEVEL YANG SUDAH DI KLASIFIKASIKAN DENGAN PROMPT AI
            })

            # TULIS SATU ROW KE FILE BATCH
            f_out.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")

        print(f"[DONE] Batch {batch_idx+1} tersimpan: {batch_file}")

print("SELESAI. Semua batch yang diperlukan sudah diproses (atau sudah ada file-nya).")

Total baris di Excel      : 1000
Effective rows (diproses) : 1000
BATCH_SIZE                : 1000
Total batch               : 1
[SKIP] /Users/irz/projects/dtp-data-pipeline/Pipeline Flagging/Data Diploy Flagged/Flagged_1000_Per_Class/dataset_multiturn/Layanan_Teknologi_Informasi_7_0_batch001.jsonl sudah ada. Lewati batch 1 (rows 0-999).
SELESAI. Semua batch yang diperlukan sudah diproses (atau sudah ada file-nya).
